## CREATING CATALOG, SCHEMA AND TABLES

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS ctl_crypto;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ctl_crypto.staging_schema;

In [0]:

silver_path = "abfss://cryptocontainer1@intellobankcryptodata.dfs.core.windows.net/silver/silver_crypto"
gold_path = "abfss://cryptocontainer1@intellobankcryptodata.dfs.core.windows.net/gold"

In [0]:
%sql
SHOW EXTERNAL LOCATIONS

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.staging_schema.raw_silver
USING DELTA
LOCATION "abfss://cryptocontainer1@intellobankcryptodata.dfs.core.windows.net/silver/silver_crypto";

In [0]:
%sql
SELECT * FROM ctl_crypto.staging_schema.raw_silver
LIMIT 5;

### NORMALISING THE GENERAL TABLE
By extracting the fact and dimension tables

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ctl_crypto.gold_schema;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.dim_roi(
  dim_roi_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  id STRING NOT NULL,
  roi_time DOUBLE,
  roi_currency STRING,
  roi_percentage DOUBLE,
  last_updated_ts TIMESTAMP
)
USING DELTA
LOCATION "abfss://cryptocontainer1@intellobankcryptodata.dfs.core.windows.net/gold/dim_roi";

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.dim_crypto
(
  dim_crypto_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  id STRING NOT NULL,
  name STRING,
  symbol STRING,
  ath DOUBLE,
  ath_change_percentage DOUBLE,
  market_cap_billions DOUBLE,
  volume_to_market_cap_ratio DOUBLE,
  last_updated_ts TIMESTAMP
)
USING DELTA
LOCATION "abfss://cryptocontainer1@intellobankcryptodata.dfs.core.windows.net/gold/dim_crypto";

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.crypto_fact
(
  id STRING NOT NULL,
  market_cap_rank BIGINT,
  max_supply DOUBLE,
  total_supply DOUBLE,
  circulating_supply DOUBLE,
  total_volume DOUBLE,
  market_cap DOUBLE,
  market_cap_change_24h DOUBLE,  -- Added missing data type
  market_cap_change_percentage_24h DOUBLE,
  price_change_24h DOUBLE,
  price_change_percentage_24h DOUBLE,
  fully_diluted_valuation DOUBLE,
  market_cap_billions DOUBLE,
  total_volume_double DOUBLE,
  high_24h DOUBLE,
  low_24h DOUBLE,
  current_price DOUBLE,
  date_partition DATE,  -- Added missing comma
  last_updated_ts TIMESTAMP,
  FOREIGN KEY (dim_roi_id) REFERENCES ctl_crypto.gold_schema.dim_roi(dim_roi_id),
  FOREIGN KEY (dim_crypto_id) REFERENCES ctl_crypto.gold_schema.dim_crypto(dim_crypto_id)
)
USING DELTA
LOCATION "abfss://cryptocontainer1@intellobankcryptodata.dfs.core.windows.net/gold/crypto_fact";


In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.crypto_fact
USING DELTA
LOCATION "abfss://cryptocontainer1@intellobankcryptodata.dfs.core.windows.net/gold/crypto_fact";


In [0]:
%sql
MERGE INTO ctl_crypto.gold_schema.dim_roi AS target
USING (
  SELECT
    id,
    roi_time,
    roi_percentage,
    roi_currency,
    last_updated_ts,
    ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
  FROM (
    SELECT *, 
    ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn 
    FROM ctl_crypto.staging_schema.raw_silver
  ) WHERE rn = 1
) AS updated_rn

ON target.id = updated_rn.id

WHEN MATCHED AND updated_rn.rn = 1 THEN 
  UPDATE SET
    target.roi_time = updated_rn.roi_time,
    target.roi_percentage = updated_rn.roi_percentage,
    target.roi_currency = updated_rn.roi_currency,
    target.last_updated_ts = updated_rn.last_updated_ts

WHEN NOT MATCHED AND updated_rn.rn = 1 THEN 
  INSERT (
    id, 
    roi_time,
    roi_percentage,
    roi_currency,        -- ✅ Correct order
    last_updated_ts
  ) 
  VALUES (
    updated_rn.id,
    updated_rn.roi_time,
    updated_rn.roi_percentage,  -- ✅ DOUBLE to roi_percentage
    updated_rn.roi_currency,    -- ✅ STRING to roi_currency
    updated_rn.last_updated_ts
  );

In [0]:
%sql
MERGE INTO ctl_crypto.gold_schema.dim_crypto AS target
USING (
  SELECT
    id,
    name,
    symbol,
    ath,
    ath_change_percentage,
    market_cap_billions,
    volume_to_market_cap_ratio,
    last_updated_ts,
    ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
  FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn FROM ctl_crypto.staging_schema.raw_silver) WHERE rn = 1
) AS updated_rn

ON target.id = updated_rn.id

WHEN MATCHED AND updated_rn.rn = 1 THEN 
  UPDATE SET
    target.name = updated_rn.name,
    target.symbol = updated_rn.symbol,
    target.ath = updated_rn.ath,
    target.ath_change_percentage = updated_rn.ath_change_percentage,
    target.market_cap_billions = updated_rn.market_cap_billions,
    target.volume_to_market_cap_ratio = updated_rn.volume_to_market_cap_ratio,
    target.last_updated_ts = updated_rn.last_updated_ts
    WHEN NOT MATCHED AND updated_rn.rn = 1 THEN 
  INSERT (
    id, 
    name,
    symbol,
    ath,
    ath_change_percentage,
    market_cap_billions,
    volume_to_market_cap_ratio,
    last_updated_ts
  ) 
  VALUES (
    updated_rn.id,
    updated_rn.name,
    updated_rn.symbol,
    updated_rn.ath,
    updated_rn.ath_change_percentage,
    updated_rn.market_cap_billions,
    updated_rn.volume_to_market_cap_ratio,
    updated_rn.last_updated_ts
  );

In [0]:
%sql
MERGE INTO ctl_crypto.gold_schema.crypto_fact AS target
USING (
  SELECT 
    id,
    market_cap_rank,
    max_supply,
    total_supply,
    circulating_supply,
    total_volume,
    market_cap,
    market_cap_change_24h,
    market_cap_change_percentage_24h,
    price_change_24h,
    price_change_percentage_24h,
    fully_diluted_valuation,
    market_cap_billions,
    total_volume_double,
    high_24h,
    low_24h,
    current_price,
    date_partition,  -- ✅ Alias to match table
    last_updated_ts,
    ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
  FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn FROM ctl_crypto.staging_schema.raw_silver) WHERE rn = 1
) AS updated_rn

ON target.id = updated_rn.id

WHEN MATCHED AND updated_rn.rn = 1 THEN 
  UPDATE SET
    target.market_cap_rank = updated_rn.market_cap_rank,
    target.max_supply = updated_rn.max_supply,
    target.total_supply = updated_rn.total_supply,
    target.circulating_supply = updated_rn.circulating_supply,
    target.total_volume = updated_rn.total_volume,
    target.market_cap = updated_rn.market_cap,
    target.market_cap_change_24h = updated_rn.market_cap_change_24h,
    target.market_cap_change_percentage_24h = updated_rn.market_cap_change_percentage_24h,
    target.price_change_24h = updated_rn.price_change_24h,
    target.price_change_percentage_24h = updated_rn.price_change_percentage_24h,
    target.fully_diluted_valuation = updated_rn.fully_diluted_valuation,
    target.market_cap_billions = updated_rn.market_cap_billions,
    target.total_volume_double = updated_rn.total_volume_double,
    target.high_24h = updated_rn.high_24h,
    target.low_24h = updated_rn.low_24h,
    target.current_price = updated_rn.current_price,
    target.date_partition = updated_rn.date_partition,  -- ✅ Fixed spelling
    target.last_updated_ts = updated_rn.last_updated_ts

WHEN NOT MATCHED AND updated_rn.rn = 1 THEN 
  INSERT (
    id,
    market_cap_rank,
    max_supply,
    total_supply,
    circulating_supply,
    total_volume,
    market_cap,
    market_cap_change_24h,
    market_cap_change_percentage_24h,
    price_change_24h,
    price_change_percentage_24h,
    fully_diluted_valuation,
    market_cap_billions,
    total_volume_double,
    high_24h,
    low_24h,
    current_price,
    date_partition,  -- ✅ Fixed spelling
    last_updated_ts
  ) 
  VALUES (
    updated_rn.id,
    updated_rn.market_cap_rank,
    updated_rn.max_supply,
    updated_rn.total_supply,
    updated_rn.circulating_supply,
    updated_rn.total_volume,
    updated_rn.market_cap,
    updated_rn.market_cap_change_24h,
    updated_rn.market_cap_change_percentage_24h,
    updated_rn.price_change_24h,
    updated_rn.price_change_percentage_24h,
    updated_rn.fully_diluted_valuation,
    updated_rn.market_cap_billions,
    updated_rn.total_volume_double,
    updated_rn.high_24h,
    updated_rn.low_24h,
    updated_rn.current_price,
    updated_rn.date_partition,  -- ✅ Fixed spelling
    updated_rn.last_updated_ts
  );

In [0]:
Top_10 = spark.sql('''
                   SELECT * FROM ctl_crypto.gold_schema.crypto_fact
                    LIMIT 10
                   ''')

In [0]:
Top_10.display()

In [0]:
transformed_data = spark.sql("""
SELECT
    dc.dim_crypto_id,
    cf.id,
    name,
    symbol,
    ath,
    ath_change_percentage,
    cf.market_cap_billions,
    volume_to_market_cap_ratio,
    (price_change_24h / cf.market_cap_billions) AS normalized_price_change_24h
FROM
    ctl_crypto.gold_schema.crypto_fact cf
JOIN
    ctl_crypto.gold_schema.dim_crypto dc
ON cf.id = dc.id
""")


In [0]:
transformed_data.display()

In [0]:
%sql
select market_cap_rank, max_supply, total_volume, market_cap
FROM ctl_crypto.gold_schema.crypto_fact
ORDER BY market_cap DESC
LIMIT 10;


In [0]:
from pyspark.sql.functions import *

binance_df = transformed_data.filter(col("id").like("%binance%"))

# Show the result
binance_df.display()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.dim_exchange_info (
  dim_exchange_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  exchange_id STRING NOT NULL,
  exchange_name STRING,
  country STRING,
  trust_score INT,
  year_established INT,
  last_updated_ts TIMESTAMP
)
USING DELTA
LOCATION "abfss://cryptocontainer1@intellobankcryptodata.dfs.core.windows.net/gold/dim_exchange_info";

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.dim_category (
  dim_category_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  category_id STRING NOT NULL,
  category_name STRING,
  description STRING,
  no_of_coins INT,
  last_updated_ts TIMESTAMP
)
USING DELTA
LOCATION "abfss://cryptocontainer1@intellobankcryptodata.dfs.core.windows.net/gold/dim_category";

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.dim_time (
  date_id INT NOT NULL,
  full_date DATE,
  day INT,
  month INT,
  year INT,
  quarter INT,
  day_name STRING,
  month_name STRING,
  is_weekend BOOLEAN
)
USING DELTA
LOCATION "abfss://cryptocontainer1@intellobankcryptodata.dfs.core.windows.net/gold/dim_time";


In [0]:
%sql
MERGE INTO ctl_crypto.gold_schema.dim_time AS target
USING (
  SELECT DISTINCT
    CAST(DATE_FORMAT(last_updated_ts, 'yyyyMMdd') AS INT) AS date_id,
    CAST(last_updated_ts AS DATE) AS full_date,
    DAY(last_updated_ts) AS day,
    MONTH(last_updated_ts) AS month,
    YEAR(last_updated_ts) AS year,
    QUARTER(last_updated_ts) AS quarter,
    DATE_FORMAT(last_updated_ts, 'EEEE') AS day_name,
    DATE_FORMAT(last_updated_ts, 'MMMM') AS month_name,
    CASE WHEN WEEKDAY(last_updated_ts) IN (5, 6) THEN TRUE ELSE FALSE END AS is_weekend
  FROM ctl_crypto.staging_schema.raw_silver
) AS source
ON target.date_id = source.date_id

WHEN MATCHED THEN 
  UPDATE SET
    target.full_date = source.full_date,
    target.day = source.day,
    target.month = source.month,
    target.year = source.year,
    target.quarter = source.quarter,
    target.day_name = source.day_name,
    target.month_name = source.month_name,
    target.is_weekend = source.is_weekend

WHEN NOT MATCHED THEN 
  INSERT (
    date_id, full_date, day, month, year, quarter, day_name, month_name, is_weekend
  ) 
  VALUES (
    source.date_id, source.full_date, source.day, source.month, source.year,
    source.quarter, source.day_name, source.month_name, source.is_weekend
  );


In [0]:
%sql
MERGE INTO ctl_crypto.gold_schema.dim_category AS target
USING (
  SELECT DISTINCT
    CASE 
      WHEN market_cap_rank <= 10 THEN 'blue_chip'
      WHEN market_cap_rank <= 100 THEN 'mid_cap'
      ELSE 'small_cap'
    END AS category_id,

    CASE 
      WHEN market_cap_rank <= 10 THEN 'Blue Chip Coins'
      WHEN market_cap_rank <= 100 THEN 'Mid Cap Coins'
      ELSE 'Small Cap Coins'
    END AS category_name,

    'Derived from market_cap_rank field' AS description,

    COUNT(*) OVER (
      PARTITION BY 
        CASE 
          WHEN market_cap_rank <= 10 THEN 'blue_chip'
          WHEN market_cap_rank <= 100 THEN 'mid_cap'
          ELSE 'small_cap'
        END
    ) AS no_of_coins,

    CURRENT_TIMESTAMP() AS last_updated_ts

  FROM ctl_crypto.staging_schema.raw_silver
  WHERE market_cap_rank IS NOT NULL
) AS source
ON target.category_id = source.category_id

WHEN MATCHED THEN 
  UPDATE SET
    target.category_name = source.category_name,
    target.description = source.description,
    target.no_of_coins = source.no_of_coins,
    target.last_updated_ts = source.last_updated_ts

WHEN NOT MATCHED THEN 
  INSERT (
    category_id, category_name, description, no_of_coins, last_updated_ts
  )
  VALUES (
    source.category_id, source.category_name, source.description,
    source.no_of_coins, source.last_updated_ts
  );


In [0]:
%sql
MERGE INTO ctl_crypto.gold_schema.dim_exchange_info AS target
USING (
  SELECT 
    'default_exchange' AS exchange_id,
    'Unknown Exchange' AS exchange_name,
    'Unknown' AS country,
    0 AS trust_score,
    0 AS year_established,
    CURRENT_TIMESTAMP() AS last_updated_ts
) AS source
ON target.exchange_id = source.exchange_id

WHEN NOT MATCHED THEN 
  INSERT (
    exchange_id, exchange_name, country, trust_score, year_established, last_updated_ts
  )
  VALUES (
    source.exchange_id, source.exchange_name, source.country,
    source.trust_score, source.year_established, source.last_updated_ts
  );
